# Practic examples of Systools - Distribution module¶

In [2]:
# importing
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from systool import data, distribution
import pandas as pd

*distribution* is a module to perform the trip distribution, one of the steps of a Four Step Model on Transport Planning. The pce of code that we use was provided buy _Chetan Joshi_ and can be revised on the distribution.py file. 

Regarding the use of the availables functions it self, let's say that you have projections over dataset saved on a projections.csv and you also decided to use the regressions of the last step.

Mind that all those are examples with toy data, so the regressions may not make sense on real life.


In [7]:
df = data.open_file('examples_databases\input_distribuicao.txt')
df.head()

,ZONAS\tEMPREGOS\tPOP\tDOMICILIOS\tPEA
0,1\t42.438\t488.294\t132.446\t460.674
1,2\t229.825\t863.829\t705.817\t137.306
2,3\t41.812\t329.147\t122.317\t89.112
3,4\t156.163\t1.010.357\t829.559\t389.087
4,5\t60.082\t416.631\t348.208\t93.815


Stay alert! Always check what you are reading and investigate to resolve the problem.

In [27]:
df = data.open_file('examples_databases\input_distribuicao.txt', kwargs={'sep':'\t', 'decimal':'.'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 0 to 118
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ZONAS       119 non-null    int64  
 1   EMPREGOS    119 non-null    float64
 2   POP         119 non-null    float64
 3   DOMICILIOS  119 non-null    float64
 4   PEA         119 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 5.6 KB


Now we need to generate our new ATRACTION and PRODUCTION vectors using our equations and projections for a future year

> regressionATRA = +0.314 * **EMPREGOS**

> regressionPROD = +0.019 * **PEA** +0.017 * **DOMICILIOS** +0.069 * **POP** +420.483

In [28]:
ATRA = 0.314 * df.EMPREGOS
PROD = 0.019 * df.PEA + 0.017 * df.DOMICILIOS +0.069 * df.POP +420.483

Now that we have new vectors we can distribute them using any methodology available...

* **CalcFratar** : Calculates a Fratar/IPF on a seed matrix given row and column (P and A) totals
* **CalcSinglyConstrained** : Calculates a singly constrained trip distribution for given P/A vectors and a friction factor matrix
* **CalcDoublyConstrained** : Calculates a doubly constrained trip distribution for given P/A vectors and a friction factor matrix (P and A should be balanced before usage, if not then A is scaled to P)
* **CalcMultiFratar** : Applies fratar model to given set of trip matrices with multiple target production vectors and one attraction vector
* **CalcMultiDistribute** : Applies gravity model to a given set of frication matrices with multiple production vectors and one target attraction vector


You can always use help(function name) to understand better the paramters


In [29]:
help(distribution.CalcFratar)

Help on function CalcFratar in module systool.distribution:

CalcFratar(ProdA, AttrA, Trips1, maxIter=10, print_balance=False)
    Calculates fratar trip distribution
    ProdA = Production target as array
    AttrA = Attraction target as array
    Trips1 = Seed trip table for fratar
    maxIter (optional) = maximum iterations, default is 10
    Returns fratared trip table



In [30]:
df = data.open_file('examples_databases\input_geracao.xlsx', kwargs={'sheet_name':'Friction'})
df_F.head()

AttributeError: 'int' object has no attribute 'startswith'

In [ ]:
 # attention! the parameters of distribution module are Numpy.Arrays not DataFrames
mtx = CalcSinglyConstrained(PROD, ATRA, df_F.values)

You can save your matrix result and keep working on your python skills or modelling!